In [10]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, SMOL_LM_135M, SFT_MATH_CONFIG

cfg = SFT_MATH_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()
wrapper.init_data_module()

2025-03-22 13:27:19.026 | INFO     | trl_wrapper.trainer_wrapper:init_model:269 - Using device: mps
2025-03-22 13:27:19.026 | INFO     | trl_wrapper.trainer_wrapper:init_model:283 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa
2025-03-22 13:27:19.753 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:304 - Using dataset path: openai/gsm8k
2025-03-22 13:27:19.753 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:309 - Using chat template override: smollmv2
2025-03-22 13:27:19.754 | INFO     | trl_wrapper.wrapper_config:__init__:127 - Cache dir: ../dataset_caches/g_s_m8_k_data_module


In [12]:
wrapper.init_trainer()

2025-03-22 13:27:24.353 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:346 - Saving output to: ./runs/03-22-13-27-839672-smollm2-135m-instruct-
2025-03-22 13:27:24.354 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:369 - Initializing trainer, run_name: 03-22-13-27-839672-smollm2-135m-instruct-, wandb project: qwen-math-sft
2025-03-22 13:27:24.355 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:440 - Using collator: <model.reasoning.ChatDataCollator object at 0x34303edd0>
/Users/brianfitzgerald/Documents/GitHub/smolmodels/notebooks/../trl_wrapper/trainer_wrapper.py:442: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `CustomSFTTrainer.__init__`. Use `processing_class` instead.
  


In [ ]:
wrapper.train()